# SCC-ICMC-USP - 1o. semestre de 2024
# SCC5871/MAI5025 - Exercício 2

### Profa. Roseli A. F. Romero


Alunos:


1.   Julyana Flores de Prá
2.   Thiago Rafael Mariotti Claudio

Para esse exercício, vamos utilizar o dataset Iris. Ele descreve atributos sobre 3 tipos de flores.
O objetivo é classificar qual o tipo de flor de acordo com os atributos disponíveis. Vamos trabalhar apenas com as duas primeiras classes para que
o problema de classificação binária.

In [10]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import log_loss
from sklearn.datasets import load_iris
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [11]:
iris = load_iris()
# Nessa primeira parte, vamos trabalhar apenas com as duas primeiras features e as duas primeiras classes
X = iris.data[:100, :4]
y = iris.target[:100]

---

### Questão 1.


- a) Utilizando o sklearn, defina uma [MLP para classificação binária](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) com a seguinte configuração:
    - função de ativação ReLU;
    - duas camadas escondidads com 10 neurônios cada;
    - taxa de aprendizado igual a 1e-2;
    - utilizando o algoritmo de otimização de gradiente descendente estocástico;
    - utilizando 10 iterações máximas (épocas);
    - use random_state=1234


- b) Treine a MLP definida no conjunto Iris simplificado definido na questçao anterior, e calcule a cross-entropy loss binária seguindo a definição a seguir.

In [12]:
# a)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

mlp = MLPClassifier(activation='relu', hidden_layer_sizes=(10, 10), learning_rate_init=1e-2, solver='sgd', max_iter=100, random_state=1234)
mlp.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(10, 10), learning_rate_init=0.01,
              max_iter=100, random_state=1234, solver='sgd')

In [13]:
# b)
y_pred_proba = mlp.predict_proba(X_test)
binary_cross_entropy_loss = log_loss(y_test, y_pred_proba)

print("Binary Cross-Entropy Loss:", binary_cross_entropy_loss)

Binary Cross-Entropy Loss: 0.0062214020615873455


---

### Questão 2.

Para avaliar os modelos que serão testados, implemente a função `evaluate_model()`. Essa função recebe um modelo de classificador genérico (`model`) e avalia sua acurácia utilizando **10-fold stratified cross-validation**, retornando a média das acurácias de cada fold. O parâmetro `X` indica os dados e `y` os labels.
- Sugestão: há duas formas de implementar a validação cruzada: treinar manualmente os modelos nos [splits gerados](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) ou utilizar a função [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) do sklearn. Atente-se ao cálculo da acurácia.

- Para garantir uma melhor performance dos algoritmos, faça o preprocessamento desses dados através da classe `sklearn.preprocessing.StandardScaler`.

In [14]:
def evaluate_model(model, X, y):
    """
    Evaluates the model using 10-fold stratified cross-validation and return the mean accuracy.
    
    Parameters:
        model: Classifier model to evaluate.
        X: Input features.
        y: Target labels.
        
    Returns:
        mean_accuracy: Mean accuracy across all folds.
    """
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)
    
    accuracies = cross_val_score(model, X_scaled, y, cv=skf, scoring='accuracy')
    
    mean_accuracy = accuracies.mean()
    
    return mean_accuracy

---

### Questão 3.

Agora para estruturar e organizar melhor nossos testes, vamos utilizar as estruturas de dicionário do Python. Por exemplo, se formos definir dois modelos de Multi-Layer Perceptron, podemos escrever:

```
experimentos = {
    "MLP camada escondida (5,)": MLPClassifier(hidden_layer_sizes=(5,),
    "MLP camada escondida (5,5)": MLPClassifier(hidden_layer_sizes=(5,5)        
}
```

Isso pode ser feito pois o Python trata funções como funções de primeira classe. Isso é, funções podem ser tratadas como variáveis.

Portanto, defina um dicionário de experimentos com ao menos 3 modelos de MLP (`sklearn.neural_network.MLPClassifier`). Para isso varie parâmetros como o número de camadas escondidas, função de ativação e número de neurônios.

- Dica: Ver documentação em https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
- Utilize um número de iterações >= 50 para garantir convergência.
- Experimente diferentes taxas de aprendizado e número máximo de iterações (épocas) de forma a garantir convergência no treino.

In [15]:
experiments = {
    "MLP camada escondida (10,)" : MLPClassifier(hidden_layer_sizes=(10,), max_iter=500, random_state=1234),
    "MLP camadas escondidas (10, 10) ReLU": MLPClassifier(hidden_layer_sizes=(10, 10), activation='relu', max_iter=500, random_state=1234),
    "MLP camadas escondidas (20, 10) Tanh": MLPClassifier(hidden_layer_sizes=(20, 10), activation='tanh', max_iter=500, random_state=1234)
}

---

### Questão 4.

- a) Para cada modelo instanciado na Questão 3, utilize a função criada na questão 3 para calcular sua acurácia. Exiba o nome do modelo e sua acurácia.
- b) Determine qual o melhor classificador dentre os especificados e justifique sua escolha.

In [16]:
# a)
results = {}
for name, model in experiments.items():
    accuracy = evaluate_model(model, X, y)
    results[name] = accuracy
    print(f"{name}: Acurácia = {accuracy:.4f}")

# pq tá todo mundo com a mesma acuracia? será que eu sou burra?

MLP camada escondida (10,): Acurácia = 1.0000
MLP camadas escondidas (10, 10) ReLU: Acurácia = 1.0000
MLP camadas escondidas (20, 10) Tanh: Acurácia = 1.0000


In [17]:
# b)
best_model = max(results, key=results.get)
acc_best_model = results[best_model]

print(f"\nO melhor classificador é '{best_model}' com uma acurácia de {acc_best_model:.4f}.")


O melhor classificador é 'MLP camada escondida (10,)' com uma acurácia de 1.0000.
